In [68]:
# Imports

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [69]:
# Loading data

RB_TRAIN_PATH = "resources/dataset_train_gp_rb"
RB_TEST_PATH = "resources/dataset_test_gp_rb"
NB_TRAIN_PATH = "resources/dataset_train_gp_nb"
NB_TEST_PATH = "resources/dataset_test_gp_nb"

def load_scores(path):
    with open(os.path.join(path, "scores.txt"), "r") as file:
        scores = [float(line.strip()) for line in file]
    return np.array(scores)

def load_maps(path, gen): 
    base_file = ''
    if gen == "rb":
        base_file = "generated_map{}.txt"
    elif gen == "nb":
        base_file = "generated_map_ca{}.txt"
    
    maps = []
    for i in range(50):
        with open(os.path.join(path, base_file.format(i)), "r") as file:
            map_data = [list(map(int, line.strip().split())) for line in file]
            maps.append(map_data)
    
    return np.array(maps)

# Load data

def data_loading(gen):
    if gen == 'rb':
        X_train, y_train = load_maps(RB_TRAIN_PATH, "rb"), load_scores(RB_TRAIN_PATH)
        X_test, y_test = load_maps(RB_TRAIN_PATH, "rb"), load_scores(RB_TRAIN_PATH)

        X_train, X_test = X_train / 3.0, X_test / 3.0
        
    elif gen == 'nb':
        X_train, y_train = load_maps(NB_TRAIN_PATH, "nb"), load_scores(NB_TRAIN_PATH)
        X_test, y_test = load_maps(NB_TRAIN_PATH, "nb"), load_scores(NB_TRAIN_PATH)

        X_train, X_test = X_train / 2.0, X_test / 2.0 
    
    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test = data_loading('nb')

# Add channel dimension
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [70]:
def create_model(input_shape):
    model = Sequential()
    
    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    
    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))  # Output layer (regression task)
    
    return model

input_shape = (12, 16, 1)
model = create_model(input_shape)

In [71]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 1s 197ms/step - loss: 20.1072 - mean_absolute_error: 3.8074 - val_loss: 13.9056 - val_mean_absolute_error: 3.0152
Epoch 2/100
2/2 [==============================] - 0s 91ms/step - loss: 17.9046 - mean_absolute_error: 3.5297 - val_loss: 12.3390 - val_mean_absolute_error: 2.7350
Epoch 3/100
2/2 [==============================] - 0s 84ms/step - loss: 15.8965 - mean_absolute_error: 3.2511 - val_loss: 10.7293 - val_mean_absolute_error: 2.4187
Epoch 4/100
2/2 [==============================] - 0s 96ms/step - loss: 13.7644 - mean_absolute_error: 2.9302 - val_loss: 8.9954 - val_mean_absolute_error: 2.2257
Epoch 5/100
2/2 [==============================] - 0s 108ms/step - loss: 11.4273 - mean_absolute_error: 2.5943 - val_loss: 7.3185 - val_mean_absolute_error: 2.0780
Epoch 6/100
2/2 [==============================] - 0s 96ms/step - loss: 8.9944 - mean_absolute_error: 2.2581 - val_loss: 6.0084 - val_mean_absolute_error: 2.0109
Epoch 7/100
2/2 [=

In [72]:
loss, mae = model.evaluate(X_test, y_test)
print(f"Validation Loss: {loss}, MAE: {mae}")

# Predict scores for the test set
predictions = model.predict(X_test)

# Compare predictions to actual values
for i in range(10):  # Display first 10 predictions
    print(f'Predicted: {predictions[i][0]}, Actual: {y_test[i]}')

2/2 [==============================] - 0s 2ms/step - loss: 1.0818 - mean_absolute_error: 0.4454
Validation Loss: 1.0817549228668213, MAE: 0.4453932046890259
2/2 [==============================] - 0s 1ms/step
Predicted: 4.983675003051758, Actual: 5.0
Predicted: 10.03385066986084, Actual: 10.0
Predicted: 3.003403663635254, Actual: 3.0
Predicted: 5.039804935455322, Actual: 5.0
Predicted: 4.003676414489746, Actual: 4.0
Predicted: 0.9824191927909851, Actual: 1.0
Predicted: 6.983707427978516, Actual: 7.0
Predicted: 0.10980422049760818, Actual: 0.0
Predicted: 3.0206947326660156, Actual: 3.0
Predicted: 3.0014028549194336, Actual: 3.0
